In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import torch
import time
import os
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist
from kymatio.torch import HarmonicScattering3D
from kymatio.scattering3d.backend.torch_backend import TorchBackend3D
from kymatio.scattering3d.utils import generate_weighted_sum_of_gaussians
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

In [2]:
import os
import csv
import numpy as np

# === PARAMÈTRES ===
data_dir = "data"
atoms_dir = os.path.join(data_dir, "atoms", "train")
energies_file = os.path.join(data_dir, "energies", "train.csv")

# === MAPPAGE DES SYMBOLES ATOMIQUES VERS NUMÉROS ATOMIQUES ===
symbol_to_number = {
    'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9,
    # tu peux en ajouter d'autres si besoin
}

# === LECTURE DES ÉNERGIES ===
energies_dict = {}
with open(energies_file, "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        mol_id = int(row["id"])
        energy = float(row["energy"])
        energies_dict[mol_id] = energy

# === EXTRACTION DES .XYZ ===
positions_list = []
charges_list = []
energies_list = []

max_atoms = 0
molecule_ids = sorted(energies_dict.keys())

for mol_id in molecule_ids:
    filename = f"id_{mol_id}.xyz"
    filepath = os.path.join(atoms_dir, filename)

    with open(filepath, "r") as f:
        lines = f.readlines()[2:]  # skip the 2 header lines

    charges = []
    positions = []

    for line in lines:
        parts = line.strip().split()
        if len(parts) == 4:
            symbol = parts[0]
            xyz = list(map(float, parts[1:]))

            charges.append(symbol_to_number.get(symbol, 0))  # 0 pour atomes inconnus
            positions.append(xyz)

    num_atoms = len(positions)
    max_atoms = max(max_atoms, num_atoms)

    charges_list.append(charges)
    positions_list.append(positions)
    energies_list.append(energies_dict[mol_id])

# === TRANSFORMATION EN TABLEAUX AVEC PADDING ===
num_molecules = len(charges_list)

positions_array = np.zeros((num_molecules, max_atoms, 3), dtype=np.float32)
charges_array = np.zeros((num_molecules, max_atoms), dtype=np.int32)

for i in range(num_molecules):
    n = len(positions_list[i])
    positions_array[i, :n, :] = positions_list[i]
    charges_array[i, :n] = charges_list[i]

energies_array = np.array(energies_list, dtype=np.float32)

# === DICTIONNAIRE FINAL ===
dataset = {
    "positions": positions_array,
    "charges": charges_array,
    "energies": energies_array
}

# ✅ Vérification
print("positions shape:", dataset["positions"].shape)
print("charges shape:", dataset["charges"].shape)
print("energies shape:", dataset["energies"].shape)
dataset


positions shape: (6591, 23, 3)
charges shape: (6591, 23)
energies shape: (6591,)


{'positions': array([[[ 0.356436, -3.423619, -0.385554],
         [ 0.674042, -1.997613, -0.81716 ],
         [-0.160963, -0.953233, -0.067053],
         ...,
         [ 0.      ,  0.      ,  0.      ],
         [ 0.      ,  0.      ,  0.      ],
         [ 0.      ,  0.      ,  0.      ]],
 
        [[-0.978257, -0.976201,  1.620806],
         [-1.408462,  0.179831,  0.72651 ],
         [-0.815131,  0.090689, -0.68896 ],
         ...,
         [ 0.      ,  0.      ,  0.      ],
         [ 0.      ,  0.      ,  0.      ],
         [ 0.      ,  0.      ,  0.      ]],
 
        [[-1.03387 ,  1.141438,  1.596581],
         [-1.355754, -0.173724,  0.899321],
         [-0.81162 , -0.234679, -0.533178],
         ...,
         [ 0.      ,  0.      ,  0.      ],
         [ 0.      ,  0.      ,  0.      ],
         [ 0.      ,  0.      ,  0.      ]],
 
        ...,
 
        [[-0.318473, -0.027506, -2.15573 ],
         [-0.710632, -0.183971, -0.682483],
         [-0.123643, -1.472825, -0.057298

# Matrice distance ordonnées

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def compute_sorted_lower_triangle_distances(coords, mask):
    """
    coords : (max_atoms, 3) coordonnées
    mask : (max_atoms,) booléen indiquant quels atomes sont réels (True) ou padding (False)
    """
    n = np.sum(mask)
    if n <= 1:
        return np.array([])  # pas assez d'atomes

    coords_real = coords[mask]

    dist_mat = np.linalg.norm(coords_real[:, None, :] - coords_real[None, :, :], axis=-1)
    lower_tri_indices = np.tril_indices(n, k=-1)
    lower_tri_values = dist_mat[lower_tri_indices]
    sorted_distances = np.sort(lower_tri_values)
    return sorted_distances

def prepare_features(positions_array, charges_array):
    n_samples = positions_array.shape[0]
    max_atoms = positions_array.shape[1]

    feature_list = []
    for i in range(n_samples):
        mask = charges_array[i] > 0  # atomes réels
        dist_vec = compute_sorted_lower_triangle_distances(positions_array[i], mask)

        charges_real = charges_array[i][mask]
        charges_sorted = np.sort(charges_real)

        # Concaténer charges triées et distances triées (padding pour uniformité)
        feature = np.concatenate([charges_sorted, dist_vec])
        feature_list.append(feature)

    # Trouver la taille max des vecteurs
    max_len = max(len(f) for f in feature_list)
    features_padded = np.zeros((n_samples, max_len), dtype=np.float32)

    for i, f in enumerate(feature_list):
        features_padded[i, :len(f)] = f

    return features_padded

# Préparer les features
X = prepare_features(dataset["positions"], dataset["charges"])
y = dataset["energies"]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Évaluation
mse = mean_squared_error(y_test, y_pred)
print(f"MSE test set: {mse**0.5:.4f}")


MSE test set: 2.3050


# Distance à l'origine 

In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# --- Fonctions pour calculer les features ---

def compute_center_of_mass(positions, charges):
    masses = charges.astype(np.float32)
    total_mass = np.sum(masses)
    if total_mass == 0:
        return np.zeros(3)
    center_of_mass = np.sum(positions * masses[:, None], axis=0) / total_mass
    return center_of_mass

def distances_to_center_of_mass(positions, charges):
    mask = charges > 0
    pos_real = positions[mask]
    charges_real = charges[mask]
    com = compute_center_of_mass(pos_real, charges_real)
    dist = np.linalg.norm(pos_real - com, axis=1)
    sorted_dist = np.sort(dist)
    return sorted_dist

def prepare_features_com_distances(positions_array, charges_array):
    n_samples = positions_array.shape[0]
    feature_list = []
    for i in range(n_samples):
        dist_vec = distances_to_center_of_mass(positions_array[i], charges_array[i])
        feature_list.append(dist_vec)
    
    max_len = max(len(f) for f in feature_list)
    features_padded = np.zeros((n_samples, max_len), dtype=np.float32)
    for i, f in enumerate(feature_list):
        features_padded[i, :len(f)] = f
    return features_padded

# --- Préparation des features ---

X = prepare_features_com_distances(dataset["positions"], dataset["charges"])
y = dataset["energies"]

# --- Split train/test ---

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Modèle de régression linéaire ---

model = LinearRegression()

model.fit(X_train, y_train)

# --- Prédictions et évaluation ---

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"MSE sur le test set: {mse**0.5:.4f}")


MSE sur le test set: 3.2969


In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# --- Préparation des features ---

X = prepare_features_com_distances(dataset["positions"], dataset["charges"])
y = dataset["energies"]

# --- Split train/test ---

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Modèle XGBoost ---

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)

# --- Prédictions et évaluation ---

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)**0.5  # racine de MSE

print(f"RMSE sur le test set: {rmse:.4f}")


RMSE sur le test set: 2.3521


In [8]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def compute_connectivity(positions, charges, threshold=1.6):
    mask = charges > 0
    pos_real = positions[mask]
    n = pos_real.shape[0]
    dist_mat = np.linalg.norm(pos_real[:, None, :] - pos_real[None, :, :], axis=-1)
    connectivity = (dist_mat < threshold) & (dist_mat > 0)
    return connectivity, mask

def compute_angles(positions, charges, threshold=1.6):
    connectivity, mask = compute_connectivity(positions, charges, threshold)
    pos_real = positions[mask]
    n = pos_real.shape[0]

    angles = []
    for b in range(n):
        neighbors = np.where(connectivity[b])[0]
        if len(neighbors) < 2:
            continue
        for i in range(len(neighbors)):
            for j in range(i+1, len(neighbors)):
                a, c = neighbors[i], neighbors[j]
                vec_ba = pos_real[a] - pos_real[b]
                vec_bc = pos_real[c] - pos_real[b]

                vec_ba /= np.linalg.norm(vec_ba)
                vec_bc /= np.linalg.norm(vec_bc)

                cos_angle = np.clip(np.dot(vec_ba, vec_bc), -1.0, 1.0)
                angle = np.arccos(cos_angle)
                angles.append(angle)

    angles = np.array(angles)
    angles_deg = np.degrees(angles)
    angles_sorted = np.sort(angles_deg)
    return angles_sorted

def prepare_features_angles(positions_array, charges_array, threshold=1.6):
    n_samples = positions_array.shape[0]
    feature_list = []
    for i in range(n_samples):
        angle_vec = compute_angles(positions_array[i], charges_array[i], threshold)
        feature_list.append(angle_vec)

    max_len = max(len(f) for f in feature_list) if feature_list else 0
    features_padded = np.zeros((n_samples, max_len), dtype=np.float32)
    for i, f in enumerate(feature_list):
        features_padded[i, :len(f)] = f
    return features_padded

# Préparation des features
X = prepare_features_angles(dataset["positions"], dataset["charges"])
y = dataset["energies"]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédictions et évaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE sur le test set: {mse:.4f}")


MSE sur le test set: 12.1790
